In [1]:
1+1

2

In [2]:
import pandas as pd 

huhu = pd.read_parquet(path="output/delta_chunks_to_index.parquet")

In [3]:
huhu.shape

(114797, 6)

In [4]:
hihi = pd.read_parquet(path="output/1_manual_law_strict.parquet")

In [5]:
hihi.shape

(2084, 7)

In [6]:
hihi.head()

,chunk_id,doc_title,doc_category,doc_url,vector_text,display_text,crawled_at
0,law_luat_hon_nhan_va_gia_dinh.txt_0,Luật Hôn nhân và Gia đình 2014,Pháp luật,local/luat_hon_nhan_va_gia_dinh.txt,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,2025-12-11T10:46:46.879494
1,law_luat_hon_nhan_va_gia_dinh.txt_1,Luật Hôn nhân và Gia đình 2014,Pháp luật,local/luat_hon_nhan_va_gia_dinh.txt,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,2025-12-11T10:46:46.879494
2,law_luat_hon_nhan_va_gia_dinh.txt_2,Luật Hôn nhân và Gia đình 2014,Pháp luật,local/luat_hon_nhan_va_gia_dinh.txt,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,2025-12-11T10:46:46.879494
3,law_luat_hon_nhan_va_gia_dinh.txt_3,Luật Hôn nhân và Gia đình 2014,Pháp luật,local/luat_hon_nhan_va_gia_dinh.txt,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,2025-12-11T10:46:46.879494
4,law_luat_hon_nhan_va_gia_dinh.txt_4,Luật Hôn nhân và Gia đình 2014,Pháp luật,local/luat_hon_nhan_va_gia_dinh.txt,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,2025-12-11T10:46:46.879494


In [7]:
huhu.head()

,chunk_id,doc_title,doc_category,vector_text,display_text,doc_url
0,Anh hùng dân tộc Việt Nam_0,Anh hùng dân tộc Việt Nam,Nhân vật lịch sử Việt Nam,Lĩnh vực: Nhân vật lịch sử Việt Nam. Chủ đề: A...,Anh hùng dân tộc Việt Nam là thuật ngữ chỉ nhữ...,https://vi.wikipedia.org/wiki/Anh_h%C3%B9ng_d%...
1,Anh hùng dân tộc Việt Nam_1,Anh hùng dân tộc Việt Nam,Nhân vật lịch sử Việt Nam,Lĩnh vực: Nhân vật lịch sử Việt Nam. Chủ đề: A...,"Đinh Tiên Hoàng, tức Đinh Bộ Lĩnh: người đánh ...",https://vi.wikipedia.org/wiki/Anh_h%C3%B9ng_d%...
2,Anh hùng dân tộc Việt Nam_2,Anh hùng dân tộc Việt Nam,Nhân vật lịch sử Việt Nam,Lĩnh vực: Nhân vật lịch sử Việt Nam. Chủ đề: A...,Tiêu chuẩn\n\n14 vị Anh hùng dân tộc Việt Nam ...,https://vi.wikipedia.org/wiki/Anh_h%C3%B9ng_d%...
3,An Dương Vương_0,An Dương Vương,Triều đại Việt Nam,Lĩnh vực: Triều đại Việt Nam. Chủ đề: An Dương...,"An Dương Vương (chữ Hán: 安陽王), tên thật là Thụ...",https://vi.wikipedia.org/wiki/An_D%C6%B0%C6%A1...
4,Nhà nước Việt Nam_0,Nhà nước Việt Nam,Nhà nước Việt Nam,Lĩnh vực: Nhà nước Việt Nam. Chủ đề: Nhà nước ...,Nhà nước Cộng hòa xã hội chủ nghĩa Việt Nam là...,https://vi.wikipedia.org/wiki/Nh%C3%A0_n%C6%B0...


In [3]:
import requests
from bs4 import BeautifulSoup
import trafilatura
import pandas as pd
import time
import random
import re
from urllib.parse import urljoin
from tqdm import tqdm
import urllib3

# [QUAN TRỌNG] Tắt cảnh báo bảo mật (SSL Warning) để crawl web chính phủ cũ
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# --- CẤU HÌNH ---
OUTPUT_FILE = "targeted_knowledge.parquet"
MAX_PAGES = 5

CAMPAIGNS = [
    {
        "name": "Tu_Lieu_Dang",
        "base_url": "https://tulieuvankien.dangcongsan.vn",
        "seed_urls": [
            "https://tulieuvankien.dangcongsan.vn/ho-chi-minh-toan-tap",
            "https://tulieuvankien.dangcongsan.vn/van-kien-tu-lieu-ve-dang"
        ],
        # Regex linh hoạt hơn: Chấp nhận mọi link con chứa từ khóa
        "link_pattern": r"(ho-chi-minh-toan-tap|van-kien-tu-lieu-ve-dang)/" 
    },
    {
        "name": "Di_San_Van_Hoa",
        "base_url": "http://dsvh.gov.vn", # Đổi sang trang Cục Di sản văn hóa (chuyên sâu hơn)
        "seed_urls": [
            "http://dsvh.gov.vn/di-tich-quoc-gia-dac-biet-1756",
            "http://dsvh.gov.vn/di-tich-quoc-gia-1757"
        ],
        "link_pattern": r"dsvh.gov.vn/.*" 
    }
]

def get_soup(url):
    """Gửi request với chế độ 'giả danh' trình duyệt thật"""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
            'Referer': 'https://google.com'
        }
        # [QUAN TRỌNG] verify=False để bỏ qua lỗi SSL
        resp = requests.get(url, headers=headers, timeout=20, verify=False)
        
        if resp.status_code == 200:
            # In ra độ dài nội dung để debug
            print(f"    ✅ Connected (Size: {len(resp.content)} bytes)")
            return BeautifulSoup(resp.content, 'html.parser')
        else:
            print(f"    ❌ Status Code: {resp.status_code}")
    except Exception as e:
        print(f"    ⚠️ Connect Error: {url} - {e}")
    return None

def extract_links_from_list(campaign, url):
    links = set()
    print(f"  🔍 Inspecting: {url}")
    soup = get_soup(url)
    
    if not soup: 
        print("    ❌ Không lấy được HTML.")
        return []

    found_count = 0
    match_count = 0
    
    # Tìm tất cả thẻ a
    all_tags = soup.find_all('a', href=True)
    print(f"    ℹ️ Tìm thấy {len(all_tags)} thẻ <a> trong trang.")

    for a in all_tags:
        href = a['href']
        full_url = urljoin(campaign['base_url'], href)
        
        # [DEBUG] In thử 3 link đầu tiên để xem format
        if found_count < 3:
            print(f"      👀 Sample link: {full_url}")
        found_count += 1

        # Lọc link
        if re.search(campaign['link_pattern'], full_url):
            if any(x in full_url for x in ['print', 'javascript', '#', 'download', 'jpg', 'png']):
                continue
            
            # Chỉ lấy link chi tiết (thường dài hơn link danh mục)
            if len(full_url) > len(url) + 5: 
                links.add(full_url)
                match_count += 1
    
    print(f"    => Lọc được {match_count} link phù hợp pattern.")
    return list(links)

def crawl_article(url, category):
    try:
        # Trafilatura cũng cần config để bỏ qua SSL
        config = trafilatura.settings.use_config()
        config.set("DEFAULT", "USER_AGENT", "Mozilla/5.0")
        
        # Download thủ công bằng requests để bypass SSL, sau đó đưa vào trafilatura
        resp = requests.get(url, verify=False, timeout=10)
        
        if resp.status_code == 200:
            data = trafilatura.extract(resp.text, output_format="json", include_comments=False)
            if data:
                import json
                j = json.loads(data)
                title = j.get('title', '')
                text = j.get('text', '')
                
                # Validation: Bỏ qua bài quá ngắn (lỗi parse)
                if len(text) < 100: return None
                
                return {
                    "doc_title": title,
                    "doc_url": url,
                    "doc_category": category,
                    "original_text": text,
                    "vector_text": f"Nguồn: {category}\nTiêu đề: {title}\nNội dung:\n{text}",
                    "display_text": f"Nguồn: {category}\nTiêu đề: {title}\nNội dung:\n{text}",
                    "chunk_id": f"crawl_{int(time.time())}_{random.randint(1000,9999)}"
                }
    except: pass
    return None

def main():
    print("🕷️ STARTING TARGETED CRAWL (V2 - SSL BYPASS)...")
    all_data = []
    
    for camp in CAMPAIGNS:
        print(f"\n🚀 Campaign: {camp['name']}")
        campaign_links = set()
        
        # 1. Quét danh sách
        for seed in camp['seed_urls']:
            links = extract_links_from_list(camp, seed)
            campaign_links.update(links)
            
            # Pagination cơ bản (Trang 2)
            # Thường web VN dùng ?page=2 hoặc /p2
            next_page = f"{seed}?page=2"
            print(f"  👉 Thử trang 2: {next_page}")
            links_p2 = extract_links_from_list(camp, next_page)
            campaign_links.update(links_p2)

        unique_links = list(campaign_links)
        print(f"  => Tổng link thu được cho {camp['name']}: {len(unique_links)}")
        
        # 2. Crawl chi tiết
        if unique_links:
            print(f"  Downloading content ({len(unique_links)} bài)...")
            for url in tqdm(unique_links):
                row = crawl_article(url, camp['name'])
                if row:
                    all_data.append(row)
                time.sleep(random.uniform(0.5, 1.0))

    # 3. Lưu
    if all_data:
        df = pd.DataFrame(all_data)
        # Loại bỏ trùng lặp nội dung
        df.drop_duplicates(subset=['doc_title'], inplace=True)
        
        df.to_parquet(OUTPUT_FILE, index=False)
        print(f"\n✅ DONE! Đã lưu {len(df)} bài viết chất lượng vào {OUTPUT_FILE}")
        
        # Preview
        print("\n👀 Ví dụ dữ liệu:")
        print(df[['doc_title', 'doc_url']].head())
    else:
        print("\n❌ Vẫn không lấy được dữ liệu nào. Hãy copy log trên để debug.")

if __name__ == "__main__":
    main()

🕷️ STARTING TARGETED CRAWL (V2 - SSL BYPASS)...

🚀 Campaign: Tu_Lieu_Dang
  🔍 Inspecting: https://tulieuvankien.dangcongsan.vn/ho-chi-minh-toan-tap
    ✅ Connected (Size: 6021 bytes)
    ℹ️ Tìm thấy 1 thẻ <a> trong trang.
      👀 Sample link: https://tulieuvankien.dangcongsan.vn/
    => Lọc được 0 link phù hợp pattern.
  👉 Thử trang 2: https://tulieuvankien.dangcongsan.vn/ho-chi-minh-toan-tap?page=2
  🔍 Inspecting: https://tulieuvankien.dangcongsan.vn/ho-chi-minh-toan-tap?page=2
    ✅ Connected (Size: 6021 bytes)
    ℹ️ Tìm thấy 1 thẻ <a> trong trang.
      👀 Sample link: https://tulieuvankien.dangcongsan.vn/
    => Lọc được 0 link phù hợp pattern.
  🔍 Inspecting: https://tulieuvankien.dangcongsan.vn/van-kien-tu-lieu-ve-dang
    ✅ Connected (Size: 6021 bytes)
    ℹ️ Tìm thấy 1 thẻ <a> trong trang.
      👀 Sample link: https://tulieuvankien.dangcongsan.vn/
    => Lọc được 0 link phù hợp pattern.
  👉 Thử trang 2: https://tulieuvankien.dangcongsan.vn/van-kien-tu-lieu-ve-dang?page=2
  🔍 In

100%|██████████| 65/65 [12:08<00:00, 11.20s/it]



✅ DONE! Đã lưu 1 bài viết chất lượng vào targeted_knowledge.parquet

👀 Ví dụ dữ liệu:
  doc_title                                            doc_url
0            http://dsvh.gov.vn/mo-cu-pho-bang-nguyen-sinh-...


In [1]:
import requests
from bs4 import BeautifulSoup
import trafilatura
import pandas as pd
import time
import random
import re
from urllib.parse import urljoin
from tqdm import tqdm
import urllib3
import hashlib
import os

# Tắt cảnh báo SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# --- CẤU HÌNH ---
MAX_PAGES = 50  # Số trang mỗi mục

# Định nghĩa lại Campaign với nguồn dễ thở hơn
CAMPAIGNS = [
    # 1. LỊCH SỬ (Đã ngon, giữ nguyên)
    {
        "name": "Nguoi_Ke_Su",
        "base_url": "https://nguoikesu.com",
        "seed_urls": [
            "https://nguoikesu.com/nhan-vat",
            "https://nguoikesu.com/dong-lich-su",
            "https://nguoikesu.com/di-tich-lich-su"
        ],
        "link_pattern": r"nguoikesu.com/(nhan-vat|dong-lich-su|di-tich-lich-su)/",
        "page_param": "?start={}", # start=0, 5, 10
        "step": 5
    },
    
    # 2. CHÍNH TRỊ (Cập nhật Seed URL)
    {
        "name": "Tuyen_Giao",
        "base_url": "https://tuyengiao.vn",
        "seed_urls": [
            "https://tuyengiao.vn/hoc-tap-va-lam-theo-loi-bac",
            "https://tuyengiao.vn/bao-ve-nen-tang-tu-tuong-cua-dang",
            "https://tuyengiao.vn/van-hoa-xa-hoi"
        ],
        # Lấy tất cả link bài viết (thường có .html hoặc không có ext nhưng nằm trong subfolder)
        "link_pattern": r"tuyengiao.vn/.*", 
        "page_param": "?page={}", 
        "step": 1
    },

    # 3. LUẬT (Thay thế VBPL bằng ChinhPhu.vn - Dễ crawl hơn)
    {
        "name": "Van_Ban_Chinh_Phu",
        "base_url": "https://vanban.chinhphu.vn",
        "seed_urls": [
            # Trang tìm kiếm văn bản mới
            "https://vanban.chinhphu.vn/?pageid=27160", 
        ],
        # Link chi tiết thường chứa 'van-ban'
        "link_pattern": r"vanban.chinhphu.vn/.*van-ban", 
        "page_param": "&page={}", 
        "step": 1
    }
]

def get_soup(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        }
        # verify=False để bypass lỗi SSL của web chính phủ
        resp = requests.get(url, headers=headers, timeout=20, verify=False)
        if resp.status_code == 200:
            return BeautifulSoup(resp.content, 'html.parser')
    except Exception as e:
        # print(f"Lỗi connect {url}: {e}")
        pass
    return None

def generate_urls(camp):
    urls = []
    for seed in camp['seed_urls']:
        urls.append(seed)
        for i in range(2, MAX_PAGES + 1):
            # Tính toán tham số phân trang
            val = i if camp.get('step', 1) == 1 else (i-1) * camp['step']
            
            if "{}" in camp['page_param']:
                param = camp['page_param'].format(val)
            else:
                param = camp['page_param'] + str(val)
                
            if "?" in seed:
                full = f"{seed}&{param.replace('?', '')}"
            else:
                full = f"{seed}{param}"
            urls.append(full)
    return urls

def extract_links(camp, url):
    soup = get_soup(url)
    if not soup: return []
    
    links = set()
    for a in soup.find_all('a', href=True):
        href = a['href']
        full = urljoin(camp['base_url'], href)
        
        # Lọc rác
        if any(x in full for x in ['#', 'jpg', 'pdf', 'doc', 'download', 'print', 'javascript', 'mailto']):
            continue
            
        if re.search(camp['link_pattern'], full):
            # Với chinhphu.vn, link phải chứa 'handle' hoặc 'chi-tiet'
            if camp['name'] == "Van_Ban_Chinh_Phu":
                if "chi-tiet" in full or "handle" in full:
                    links.add(full)
            else:
                # Các trang khác: độ dài URL phải đủ lớn (tránh link trang chủ)
                if len(full) > len(camp['base_url']) + 10:
                    links.add(full)
    return list(links)

def crawl_content(url, category):
    try:
        # Dùng requests tải trước để bypass SSL
        resp = requests.get(url, verify=False, timeout=15, headers={'User-Agent': 'Mozilla/5.0'})
        if resp.status_code != 200: return None
        
        # Parse
        data = trafilatura.extract(resp.text, output_format="json", include_comments=False)
        if data:
            import json
            j = json.loads(data)
            title = j.get('title', '').strip()
            text = j.get('text', '').strip()
            
            if len(text) < 200: return None
            
            doc_hash = hashlib.md5(url.encode()).hexdigest()[:10]
            
            return {
                "chunk_id": f"{category}_{doc_hash}",
                "doc_title": title,
                "doc_url": url,
                "doc_category": category,
                "vector_text": f"Nguồn: {category}\nTiêu đề: {title}\nNội dung:\n{text}",
                "display_text": f"Title: {title}\nSource: {url}\n\n{text}"
            }
    except: pass
    return None

def main():
    print("🚀 BẮT ĐẦU CRAWL (FIXED VERSION)...")
    
    total_collected = 0
    
    for camp in CAMPAIGNS:
        print(f"\n📂 Đang xử lý nguồn: {camp['name']}")
        list_urls = generate_urls(camp)
        
        # 1. Quét Link
        article_links = set()
        print(f"   -> Đang quét {len(list_urls)} trang danh mục...")
        for l_url in tqdm(list_urls, desc="Scanning"):
            found = extract_links(camp, l_url)
            article_links.update(found)
        
        print(f"   => Tìm thấy {len(article_links)} link bài viết.")
        if len(article_links) == 0:
            print("   ⚠️ Không tìm thấy link nào. Bỏ qua nguồn này.")
            continue

        # 2. Tải nội dung
        rows = []
        for url in tqdm(list(article_links), desc="Downloading"):
            row = crawl_content(url, camp['name'])
            if row:
                rows.append(row)
            # Sleep nhẹ để không bị ban
            time.sleep(0.2) 
            
        # 3. LƯU NGAY LẬP TỨC (Save Checkpoint)
        if rows:
            filename = f"crawl_{camp['name']}.parquet"
            df = pd.DataFrame(rows)
            df.drop_duplicates(subset=['doc_url'], inplace=True)
            df.to_parquet(filename, index=False)
            print(f"   💾 Đã lưu {len(df)} bài vào '{filename}'")
            total_collected += len(df)
        else:
            print("   ❌ Không tải được nội dung bài nào.")

    print(f"\n🎉 TỔNG KẾT: Đã thu thập {total_collected} bài viết mới.")
    print("👉 Hãy thêm các file 'crawl_*.parquet' vào build_bm25_incremental.py")

if __name__ == "__main__":
    main()

🚀 BẮT ĐẦU CRAWL (FIXED VERSION)...

📂 Đang xử lý nguồn: Nguoi_Ke_Su
   -> Đang quét 150 trang danh mục...


Scanning: 100%|██████████| 150/150 [03:41<00:00,  1.47s/it]


   => Tìm thấy 453 link bài viết.


Downloading: 100%|██████████| 453/453 [11:33<00:00,  1.53s/it]


   ❌ Không tải được nội dung bài nào.

📂 Đang xử lý nguồn: Tuyen_Giao
   -> Đang quét 150 trang danh mục...


Scanning: 100%|██████████| 150/150 [00:00<00:00, 204.55it/s]


   => Tìm thấy 0 link bài viết.
   ⚠️ Không tìm thấy link nào. Bỏ qua nguồn này.

📂 Đang xử lý nguồn: Van_Ban_Chinh_Phu
   -> Đang quét 50 trang danh mục...


Scanning: 100%|██████████| 50/50 [00:52<00:00,  1.05s/it]

   => Tìm thấy 0 link bài viết.
   ⚠️ Không tìm thấy link nào. Bỏ qua nguồn này.

🎉 TỔNG KẾT: Đã thu thập 0 bài viết mới.
👉 Hãy thêm các file 'crawl_*.parquet' vào build_bm25_incremental.py
